In [74]:
from deeplotx import SoftmaxRegression, LongformerEncoder
from util import NUM_CLASSES
lf_encoder = LongformerEncoder(model_name_or_path='severinsimmler/xlm-roberta-longformer-base-16384')

[DEBUG] 2025-08-03 00:23:26,988 deeplotx.embedding : LongformerEncoder initialized on device: cuda.


In [75]:
import torch
from deeplotx.util import sha256
from vortezwohl.cache import LRUCache

CACHE = LRUCache(capacity=16384)

def encode(text: str) -> torch.Tensor:
    key = sha256(text)
    if key in CACHE:
        return CACHE[key]
    emb = lf_encoder.encode(text, cls_only=False).mean(dim=-2, dtype=model.dtype)
    CACHE[key] = emb
    return emb

In [76]:
import os
import pandas as pd
base_path = './data/multilingual_wikineural'
languages = ['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ru']
train_data = dict()
valid_data = dict()
test_data = dict()
for lang in languages:
    train_data[lang] = pd.read_csv(os.path.join(base_path, f'train_{lang}.csv')).to_dict(orient='records')
    valid_data[lang] = pd.read_csv(os.path.join(base_path, f'val_{lang}.csv')).to_dict(orient='records')
    test_data[lang] = pd.read_csv(os.path.join(base_path, f'test_{lang}.csv')).to_dict(orient='records')

In [77]:
total_train_data = []
total_valid_data = []
total_test_data = []
for lang, data in train_data.items():
    for d in data:
        total_train_data.append((d['tokens'], d['ner_tags'], d['lang']))
for lang, data in valid_data.items():
    for d in data:
        total_valid_data.append((d['tokens'], d['ner_tags'], d['lang']))
for lang, data in test_data.items():
    for d in data:
        total_test_data.append((d['tokens'], d['ner_tags'], d['lang']))

total_train_data[:2], total_valid_data[:2], total_test_data[:2]

([("['Dieses' 'wiederum' 'basierte' 'auf' 'dem' 'gleichnamigen' 'Roman' 'von'\n 'Noël' 'Calef' '.']",
   '[0 0 0 0 0 0 0 0 1 2 0]',
   'de'),
  ("['Auf' 'Helgoland' 'starben' '2014' 'sieben' 'Jungvögel' 'und' 'fünf'\n 'Altvögel' 'als' 'Verstrickungsopfer' '.']",
   '[0 5 0 0 0 0 0 0 0 0 0 0]',
   'de')],
 [("['Die' 'Europameisterschaften' 'in' 'Dresden' 'schloss' 'sie' 'mit' 'dem'\n 'zweiten' 'Platz' 'in' 'der' 'Gesamtwertung' 'ab' '.']",
   '[0 0 0 5 0 0 0 0 0 0 0 0 0 0 0]',
   'de'),
  ('[\'Die\' \'Europameisterschaften\' \'im\' \'selben\' \'Jahr\' \'in\' \'Heerenveen\'\n \'konnte\' \'sie\' \'mit\' \'dem\' \'Gewinn\' \'von\' \'vier\' \'Goldmedaillen\' \'(\'\n \'1000\' \'"\' \'m\' \',\' \'1500\' \'"\' \'m\' \',\' \'3000\' \'"\' \'m\' \',\' \'Gesamtwertung\'\n \')\' \'deutlich\' \'für\' \'sich\' \'entscheiden\' \'.\']',
   '[0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]',
   'de')],
 [("['Er' 'erklärte' ',' 'dass' 'der' 'Allgemeine' 'Nationalkongress'\n 'illega

In [79]:
from random import shuffle

train_dataset = []
valid_dataset = []
test_dataset = []
max_other_token_count = 55_000  # 55k
other_token_count_for_each_lang = {
    'de': 0,
    'en': 0,
    'es': 0,
    'fr': 0,
    'it': 0,
    'nl': 0,
    'pl': 0,
    'pt': 0,
    'ru': 0
}

def token_decode(s: str) -> list:
    return [_.strip().replace("'", '').replace('[', '').replace(']', '') for _ in s.replace("' '", '||').replace('\n', '||').split('||')]

def label_decode(s: str) -> list:
    _labels = []
    label_lists = token_decode(s)
    for label_ls in label_lists:
        _labels.extend([int(_) for _ in label_ls.split(' ')])
    return _labels

for tokens, labels, lang in total_train_data:
    tokens, labels = token_decode(tokens), label_decode(labels)
    for i, token in enumerate(tokens):
        if labels[i] > 0:
            train_dataset.append((token, labels[i]))
        else:
            if other_token_count_for_each_lang[lang] < max_other_token_count:
                other_token_count_for_each_lang[lang] += 1
                train_dataset.append((token, labels[i]))

for tokens, labels, lang in total_valid_data:
    tokens, labels = token_decode(tokens), label_decode(labels)
    for i, token in enumerate(tokens):
        if labels[i] > 0:
            valid_dataset.append((token, labels[i]))
        else:
            if other_token_count_for_each_lang[lang] < max_other_token_count:
                other_token_count_for_each_lang[lang] += 1
                valid_dataset.append((token, labels[i]))
                
                
for tokens, labels, lang in total_test_data:
    tokens, labels = token_decode(tokens), label_decode(labels)
    for i, token in enumerate(tokens):
        if labels[i] > 0:
            test_dataset.append((token, labels[i]))
        else:
            if other_token_count_for_each_lang[lang] < max_other_token_count:
                other_token_count_for_each_lang[lang] += 1
                test_dataset.append((token, labels[i]))

print('Dataset initialized', list(zip(train_dataset[:23], train_dataset[:23])))
shuffle(train_dataset)
shuffle(valid_dataset)
shuffle(test_dataset)
print('Dataset shuffled', list(zip(train_dataset[:23], train_dataset[:23])))
print(f'other_token_count_for_each_lang: {other_token_count_for_each_lang}')
f'{len(train_dataset)} tokens to train in total.'

Dataset initialized [(('Dieses', 0), ('Dieses', 0)), (('wiederum', 0), ('wiederum', 0)), (('basierte', 0), ('basierte', 0)), (('auf', 0), ('auf', 0)), (('dem', 0), ('dem', 0)), (('gleichnamigen', 0), ('gleichnamigen', 0)), (('Roman', 0), ('Roman', 0)), (('von', 0), ('von', 0)), (('Noël', 1), ('Noël', 1)), (('Calef', 2), ('Calef', 2)), (('.', 0), ('.', 0)), (('Auf', 0), ('Auf', 0)), (('Helgoland', 5), ('Helgoland', 5)), (('starben', 0), ('starben', 0)), (('2014', 0), ('2014', 0)), (('sieben', 0), ('sieben', 0)), (('Jungvögel', 0), ('Jungvögel', 0)), (('und', 0), ('und', 0)), (('fünf', 0), ('fünf', 0)), (('Altvögel', 0), ('Altvögel', 0)), (('als', 0), ('als', 0)), (('Verstrickungsopfer', 0), ('Verstrickungsopfer', 0)), (('.', 0), ('.', 0))]
Dataset shuffled [(('Albrecht', 1), ('Albrecht', 1)), (('Got', 8), ('Got', 8)), (('en', 8), ('en', 8)), (('União', 6), ('União', 6)), (('sich', 0), ('sich', 0)), (('Vrančić', 2), ('Vrančić', 2)), (('Нью-Уэстминстер', 5), ('Нью-Уэстминстер', 5)), (('en

'2787832 tokens to train in total.'

## 训练

In [92]:
model = SoftmaxRegression(input_dim=768, output_dim=NUM_CLASSES, num_heads=4, num_layers=3, expansion_factor=1.25, bias=True, dropout_rate=0.2, head_layers=2)
print(model)

Model_Name: SoftmaxRegression
In_Features: 768
Out_Features: 9
Device: cuda
Dtype: torch.float32
Total_Parameters: 49642017
Trainable_Parameters: 49642017
NonTrainable_Parameters: 0
-------------------------------
SoftmaxRegression(
  (multi_head_ffn_layers): ModuleList(
    (0-2): 3 x MultiHeadFeedForward(
      (expand_proj): Linear(in_features=768, out_features=3072, bias=True)
      (ffn_heads): ModuleList(
        (0-3): 4 x FeedForward(
          (ffn_layers): ModuleList(
            (0-1): 2 x FeedForwardUnit(
              (up_proj): Linear(in_features=768, out_features=960, bias=True)
              (down_proj): Linear(in_features=960, out_features=768, bias=True)
              (parametric_relu): PReLU(num_parameters=1)
              (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
            )
          )
        )
      )
      (out_proj): Linear(in_features=3072, out_features=768, bias=True)
    )
  )
  (out_proj): Linear(in_features=768, out_features=9, 

In [ ]:
# train
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter

train_step = 0
valid_step = 0
writer = SummaryWriter()

acc_train_loss = 0.
acc_valid_loss = 0.
eval_interval = 2000
log_interval = 200
valid_log_interval = 50

In [ ]:
from random import randint
from util import one_hot

elastic_net_param = {
    'alpha': 2e-4,
    'rho': 0.2
}
learning_rate = 2e-6
num_epochs = 1500
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    model.train()
    for i, (_token, _label) in enumerate(train_dataset):
        _one_hot_label = one_hot(_label).to(model.dtype).to(model.device)
        outputs = model.forward(encode(_token))
        loss = loss_function(outputs, _one_hot_label) + model.elastic_net(alpha=elastic_net_param['alpha'], rho=elastic_net_param['rho'])
        acc_train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if train_step % log_interval == 0 and train_step > 0:
            writer.add_scalar('train/loss', acc_train_loss / log_interval, train_step)
            print(f'- Train Step {train_step} Loss {acc_train_loss / log_interval} \\'
                  f'\nToken={_token}'
                  f'\nPred={outputs.tolist()}'
                  f'\nLabel={_one_hot_label.tolist()}', flush=True)
            acc_train_loss = 0.
        train_step += 1
        if train_step % eval_interval == 0:
            model.eval()
            rand_idx = randint(0, len(valid_dataset) - 501)
            with torch.no_grad():
                for _i, (__token, __label) in enumerate(valid_dataset[rand_idx: rand_idx + 500]):
                    _one_hot_label = one_hot(__label).to(model.dtype).to(model.device)
                    outputs = model.forward(encode(__token))
                    loss = loss_function(outputs, _one_hot_label)
                    acc_valid_loss += loss.item()
                    if valid_step % valid_log_interval == 0 and valid_step > 0:
                        writer.add_scalar('valid/loss', acc_valid_loss / log_interval, valid_step)
                        print(f'- Valid Step {valid_step} Loss {acc_valid_loss / log_interval} \\'
                              f'\nToken={__token}'
                              f'\nPred={outputs.tolist()}'
                              f'\nLabel={_one_hot_label.tolist()}', flush=True)
                        acc_valid_loss = 0.
                    valid_step += 1
            model.train()

In [ ]:
model.eval()
test_tokens = ['Mike', 'John', 'Smith', 'London', 'NYC', 'HongKong', 'China', 'South Africa', 'Korea']
with torch.no_grad():
    for _tok in test_tokens:
        _dist = model.forward(encode(_tok))
        print(f'Token={_tok}, Class={torch.argmax(_dist)}', flush=True)
model.train()
'Test finished.'